In [ ]:
# %pip install opencv-python-headless
# %pip install numpy
# %pip install matplotlib
# %pip install ipympl
# %pip install tqdm
# %pip install optv
# %pip install imageio

In [ ]:
# %load makeData_Alex
# import os, sys, cv2, random
import imageio.v3 as iio
import os
from pathlib import Path
import numpy as np

from tqdm import tqdm

from optv.parameters import ControlParams, VolumeParams
from optv.calibration import Calibration
from optv.imgcoord import image_coordinates
from optv.transforms import convert_arr_metric_to_pixel
import matplotlib.pyplot as plt


In [ ]:

#%%
class Parameter():
    # selected number of particles
    N_particles = 949
    # cameras used
    cams = [0,1,2,3]
    # resolution of the camera images
    # x_res , y_res, bit = 800, 800, 8
    x_res , y_res, bit = 2560, 2160, 8
    # camera image parameters
    # I0, I1, dI, SNR, sigma = 20000, 22000, 5, 7, 0.7
    I0, I1, dI, SNR, sigma = int(40000/256), int(44000/256), 5, 7, 0.7
#%% 


# change this function to load your calibration and perform backprojection on camera ci:
# xy_i=F(XYZ,ci)
# return array of shape (949,2) 
# 949 is the number of particles in data sets
# 2 denotes (x,y) coordiantes of the images


def Project3Dto2D(XYZ,cal,cpar):
    # print(f"Projecting {XYZ.shape} points")
    targets = convert_arr_metric_to_pixel(
        image_coordinates(XYZ, cal, cpar.get_multimedia_params()),
    cpar,
    )
    # print(f"Projecting {XYZ.shape[0]} points")
    # print(f'targets shape: {targets.shape}')
    return targets
    # return 1200*np.ones([949,2])

def Gauss(x,y,I,xmean,ymean,sigma):
    # Gauss function to blur a particle on an image
    X = (x-xmean) / sigma
    Y = (y-ymean) / sigma
    return I * np.exp( -0.5*(X**2+Y**2) ) / (2*np.pi*sigma**2)

# def main():
# load config parameter
params = Parameter()

n_cams = len(params.cams)

cpar = ControlParams(n_cams)
cpar.read_control_par("./parameters/ptv.par")

vpar = VolumeParams()
vpar.read_volume_par("./parameters/criteria.par")

# Calibration initial guess 

cals = []
for i_cam in range(n_cams):
    cal = Calibration()
    tmp = cpar.get_cal_img_base_name(i_cam)
    cal.from_file(tmp + ".ori", tmp + ".addpar")
    print(cal.get_pos(), cal.get_angles())
    cals.append(cal)
    
    
# create an intensity value for each particle ID in List 
Intensities = np.random.uniform(params.I0,params.I1,params.N_particles)

# main loop
files = Path("./origin").rglob('*.txt')

# write out final origin file
if not os.path.exists("./origin_new"):
    os.makedirs("./origin_new")
if not os.path.exists("./camera_images"):
    os.makedirs("./camera_images")


In [ ]:
# Main loop over all files
display = False

files = sorted(files, key=lambda x: int(x.stem.split('_')[-1]))
for t, filename in enumerate(tqdm(files, desc='Creating Images: ')):
    print(f"Processing file: {filename}, number: {t}")
    data = np.loadtxt(filename,skiprows=1, usecols=(0,1,2,3))
    XYZ = data[:,1:]

    # Find the mean of XYZ in each direction
    mean_XYZ = XYZ.mean(axis=0)
    # # Shift the 3D data to the origin by subtracting the mean
    # XYZ -= mean_XYZ

    xy = [Project3Dto2D(XYZ, cals[ci], cpar) for ci in params.cams]

    if display:
    # Plot the 2D projections for each camera
        plt.figure(figsize=(10, 10))
        for ci, xy_i in enumerate(xy):
            plt.subplot(2, 2, ci + 1)
            plt.scatter(xy_i[:, 0], xy_i[:, 1], s=5, alpha=0.6)
            plt.title(f'2D Projection on Camera {params.cams[ci]}')
            plt.xlabel('x [px]')
            plt.ylabel('y [px]')
            plt.xlim(0, params.x_res)
            plt.ylim(0, params.y_res)
            plt.gca().invert_yaxis()
        plt.tight_layout()
        plt.show()


    # # Shift all projected coordinates so that their mean is at the center of the image (params.x_res/2, params.y_res/2)
    # for i in range(len(xy)):
    #     mean_x = np.mean(xy[i][:, 0])
    #     mean_y = np.mean(xy[i][:, 1])
    #     shift_x = params.x_res / 2 - mean_x
    #     shift_y = params.y_res / 2 - mean_y
    #     xy[i][:, 0] += shift_x
    #     xy[i][:, 1] += shift_y
    
    for xy_i in xy:
        data = np.append(data,xy_i,axis=1)
        np.savetxt("./origin_new/"+filename.name, data, header='ID, X, Y, Z, xc0, yc0, xc1, yc1, xc2, yc2, xc3, yc3')

    # create camera images
    for xy_i, cam in zip(xy, params.cams):
        img = np.zeros((params.y_res,params.x_res))
        # go over each particle and create the blob and put it on the image
        for n in range(params.N_particles):
            px, py = xy_i[n]
            x = np.array([ np.rint(px)-2, np.rint(px)-1,  np.rint(px),   np.rint(px)+1,  np.rint(px)+2,
                            np.rint(px)-2, np.rint(px)-1,  np.rint(px),   np.rint(px)+1,  np.rint(px)+2, 
                            np.rint(px)-2, np.rint(px)-1,  np.rint(px),   np.rint(px)+1,  np.rint(px)+2, 
                            np.rint(px)-2, np.rint(px)-1,  np.rint(px),   np.rint(px)+1,  np.rint(px)+2,
                            np.rint(px)-2, np.rint(px)-1,  np.rint(px),   np.rint(px)+1,  np.rint(px)+2,]).astype(int)
            y = np.array([ np.rint(py),   np.rint(py),   np.rint(py),   np.rint(py),   np.rint(py),  
                            np.rint(py)+1, np.rint(py)+1, np.rint(py)+1, np.rint(py)+1, np.rint(py)+1,
                            np.rint(py)+2, np.rint(py)+2, np.rint(py)+2, np.rint(py)+2, np.rint(py)+2,
                            np.rint(py)-1, np.rint(py)-1, np.rint(py)-1, np.rint(py)-1, np.rint(py)-1,
                            np.rint(py)-2, np.rint(py)-2, np.rint(py)-2, np.rint(py)-2, np.rint(py)-2]).astype(int)
            intensity = Intensities[n]*np.random.normal(1,params.dI/100)
            # calculate the blob intenity distribution, the sum over I_blob is equal to intensity
            I_blob = np.rint(Gauss(x, y, intensity, px, py, params.sigma))
            for i in range(len(x)):
                img[y[i], x[i]] += I_blob[i]
        # add noise
        
        scale = np.sqrt(np.mean(img.astype(float)[img>0]**2)/params.SNR)
        # print(f"scale: {scale}, SNR: {params.SNR}")

        noise = np.random.uniform(0, scale, size=(params.y_res, params.x_res))
        img = img + noise
        img = np.clip(img, 0, 255).astype(np.uint8)

        # Display the image
        if display:
            plt.figure()
            plt.imshow(img, cmap='gray')
            plt.show()


        iio.imwrite(f"./camera_images/cam{cam}_{str(t).zfill(5)}.tif", img)
            # cv2.imwrite("./camera_images/c{cam}/c{cam}_{time}.tif".format(cam=cam,time=str(t).zfill(5)),img)
# if __name__ == "__main__":


In [ ]:
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(XYZ[:, 0], XYZ[:, 1], XYZ[:, 2], s=5, alpha=0.6)

# Set axis labels
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

# Set limits to show the volume
ax.set_xlim(XYZ[:, 0].min(), XYZ[:, 0].max())
ax.set_ylim(XYZ[:, 1].min(), XYZ[:, 1].max())
ax.set_zlim(XYZ[:, 2].min(), XYZ[:, 2].max())

ax.set_title('3D Visualization of Analysed Volume')
plt.show()

In [ ]:
mean_XYZ

### we can use the same code to create calibration images

In [ ]:
# Read calibration dot positions from file
calib_data = np.loadtxt("target_on_a_side.txt", skiprows=1, usecols=(0,1,2,3))
XYZ_calib = calib_data[:, 1:]
XYZ_calib -= XYZ_calib.mean(axis=0)  # Center the calibration points at the origin
XYZ_calib += mean_XYZ  # Center the calibration points at the origin

# Create artificial 3D calibration body: 5 rows x 5 columns x 3 planes (z), centered at (0,0,0)
# n_rows = 5
# n_cols = 15
# n_planes = 3

# x_vals = np.linspace(-30, 30, n_cols)
# y_vals = np.linspace(-30, 30, n_rows)
# z_vals = np.linspace(-30, 30, n_planes)

# # Generate grid of points
# X, Y, Z = np.meshgrid(x_vals, y_vals, z_vals, indexing='ij')
# XYZ_calib = np.column_stack([X.ravel(), Y.ravel(), Z.ravel()])
# XYZ_calib += mean_XYZ
# print(f"Artificial calibration body shape: {XYZ_calib.shape}")

# 3D plot of artificial calibration body
fig_calib = plt.figure(figsize=(8, 6))
ax_calib = fig_calib.add_subplot(111, projection='3d')
ax_calib.scatter(XYZ_calib[:, 0], XYZ_calib[:, 1], XYZ_calib[:, 2], s=30, c='r', alpha=0.7)
ax_calib.set_xlabel('X')
ax_calib.set_ylabel('Y')
ax_calib.set_zlabel('Z')
ax_calib.set_title('3D Visualization of Artificial Calibration Body')
plt.show()


In [ ]:

# Project calibration dots to each camera
xy_calib = [Project3Dto2D(XYZ_calib, cals[ci], cpar) for ci in params.cams]
# # Center the projected coordinates in the image
# for i in range(len(xy_calib)):
#     mean_x = np.mean(xy_calib[i][:, 0])
#     mean_y = np.mean(xy_calib[i][:, 1])
#     shift_x = params.x_res / 2 - mean_x
#     shift_y = params.y_res / 2 - mean_y
#     xy_calib[i][:, 0] += shift_x
#     xy_calib[i][:, 1] += shift_y

# Plot 2D scatter of calibration dots for each camera
for ci, xy_i in enumerate(xy_calib):
    plt.figure(figsize=(5, 5))
    plt.scatter(xy_i[:, 0], xy_i[:, 1], s=30, alpha=0.7)
    plt.title(f'2D Projection of Calibration Dots on Camera {params.cams[ci]}')
    plt.xlabel('x [px]')
    plt.ylabel('y [px]')
    plt.xlim(0, params.x_res)
    plt.ylim(0, params.y_res)
    plt.gca().invert_yaxis()
    plt.show()

# Create calibration images
for xy_i, cam in zip(xy_calib, params.cams):
    img = np.zeros((params.y_res, params.x_res))
    for n in range(XYZ_calib.shape[0]):
        px, py = xy_i[n]
        x = np.array([ np.rint(px)-2, np.rint(px)-1,  np.rint(px),   np.rint(px)+1,  np.rint(px)+2,
                        np.rint(px)-2, np.rint(px)-1,  np.rint(px),   np.rint(px)+1,  np.rint(px)+2, 
                        np.rint(px)-2, np.rint(px)-1,  np.rint(px),   np.rint(px)+1,  np.rint(px)+2, 
                        np.rint(px)-2, np.rint(px)-1,  np.rint(px),   np.rint(px)+1,  np.rint(px)+2,
                        np.rint(px)-2, np.rint(px)-1,  np.rint(px),   np.rint(px)+1,  np.rint(px)+2,]).astype(int)
        y = np.array([ np.rint(py),   np.rint(py),   np.rint(py),   np.rint(py),   np.rint(py),  
                        np.rint(py)+1, np.rint(py)+1, np.rint(py)+1, np.rint(py)+1, np.rint(py)+1,
                        np.rint(py)+2, np.rint(py)+2, np.rint(py)+2, np.rint(py)+2, np.rint(py)+2,
                        np.rint(py)-1, np.rint(py)-1, np.rint(py)-1, np.rint(py)-1, np.rint(py)-1,
                        np.rint(py)-2, np.rint(py)-2, np.rint(py)-2, np.rint(py)-2, np.rint(py)-2]).astype(int)
        intensity = 250  # Use a fixed intensity for calibration dots
        I_blob = np.rint(Gauss(x, y, intensity, px, py, params.sigma))
        for i in range(len(x)):
            if 0 <= y[i] < params.y_res and 0 <= x[i] < params.x_res:
                img[y[i], x[i]] += I_blob[i]
    # Add noise
    # scale = np.sqrt(np.mean(img.astype(float)[img>0]**2)/params.SNR) if np.any(img>0) else 1
    # noise = np.random.uniform(0, scale, size=(params.y_res, params.x_res))
    # img = img + noise
    img = np.clip(img, 0, 255).astype(np.uint8)

    plt.figure()
    plt.imshow(img, cmap='gray')
    plt.show()
    
    iio.imwrite(f"./camera_images/calib_cam{cam}.tif", img)